# Time and Ordering

Being fast or slow in a distributed system breaks your ability to coordinate. 

Too slow, and you miss your communication window (Operations fail because they worked on incorrect assumptions as to the state of the system). Also, you can end up introducing inconsistency into other machines, causing further break-down.

Too fast, and you end up waiting for everyone else (poor notion of Fairness).

## The Problem

Clocks aren't shared between systems. 
Internet-based systems are _asynchronous_ by nature
This is in contrast to a multi-processing system
    ? Does that apply to grid-computing systems as well?
    
## Glossary
__events__ Occurence of an action
__action__ Change of state

We'd like to view __Events__ across processes _linearly_

__Clock Skew__ Relative difference in clock values of two processes
    Distance between two clocks
__Clock Drift__ Relative difference in clock _frequencies_ of two processes
    Difference in speed between two clocks
    
If clock drift is non-zero, the clock skew will become non-zero

### Synchronizing

We define the "correct" time in terms of UTC

__Maximum Drift Rate__ How bad a clock is drifting

The worst drift between two clocks can be is 2 * MDR
    * Given two clocks with the same MDR
        * One clock is UTC + MDR
        * Once clock is UTC - MDR
        
We need to synchronize _at least_ M / 2(*MDR)
    * Time = Distance / Speed
    
#### External Syncrhonization
* Use an external, dedicated clock (called _S_)
* Ensures all system clocks are within a bound _D_ of difference to _S_
* Christian algo, NTP

#### Internal Synchronization
* Every pair of processes within group is within bound _D_
* Berkeley algo

__External Sync w/ a given bound _D_ implies Internal Sync w/ 2(*D*)__

All pairs of processes in an externally sync'd system will be within 2(*D*) of each other.

__BUT__

Interal Sync does _not_ imply External Sync

In fact, an internally sync'd system runs the risk of _all_ clocks in the system consistenly driting away from UTC.

## Synchronization Algorithms

### External Synchronization: Cristian's Algorithm

Process _P_ asks _S_ for time
_S_ responds
_P_ receives and updates

#### What's the Problem?
By the _P_ gets _S_'s time, the time's outdated!

_P_ can compensate by measuring the round-trip time of the message,
and calculating the error in the received time.

![](img/external_sync_latency.png)

**min1** Time to go from _P_ -> _S_
**min2** Time to go from _S_ -> _P_
**RTT** Time from start to reception of message

We know that the received time _T_ will _actually_ be somewhere between [_T_ + _min2_, _T_ + _RTT_ - _min1_]

* _T_ + _min2_: Only add the time required to _get_ messages from _S_
* _T_ + _RTT_ - _min1_: Sometime lost on _S_, waiting for a message to be sent

Cristians' Algo uses the halfway point:

$$
P_{T} = T + \frac{RTT + min_{2} - min_{1}}{2}
$$

#### Gotchas

* Never lower your clock time
    * Could violate your interal event ordering
* Can tweak _speed_ of clock
* If error is getting high, average multiple readings 

### NTP (Network Time Protocol)

![NTP Tree](img/ntp_hierarchy.png)

__Primary server__ Master clock
__Client__ Leaf of tree

![NTP Syncing](img/ntp_sync.png)

Offset between child process c, and parent process p:

$$
o = \frac{t_{r,1} - t_{r,2} + t_s{2} - t_{s,1}}{2}
$$

Suppose real offset is $o_{real}$

* Child is ahead of parent by $o_{real}$
* Parent is thus behind child by $-o_{real}$

$L_{1}, L_{2} = $ latency of message 1, message 2 - These are unknown!

Then,
$$
t_{r,1} = t_{s,1} + L_{1} + o_{real} \\
t_{r,2} = t_{s,2} + L_{2} - o_{real}
$$

Which breaks down to:

```
Time child recv'd Message 1  = \
    Time parent sent Message 1 + 
    Latency going from Parent => Child + 
    how far ahead in time Child is compared to parent
```
and so forth for $t_{r,2}$.

So the pair of messages (child, parent) & (parent, child) allow us to get a feel for latency between systems, as well as the clock offset.

__And yet...__
We can't get rid of error as long as latencies are non-zero

### Lamport Timestamps

__What if__ we used _relative_ timestamps instead of _absolute_ ones?
* __NTP__ used _absolute_ and we still got error
* We need our _relative_ timestamps to obey _causality_
    * Timestamp _A_ must be before _B_ if _A_ occurs before _B_
* We forsake absolutely correct timestamps in favor of relatively correct timing.

We need to define a _Happens-Before_ relationship, denoted as $\rightarrow$

* Transitive

    If $a \rightarrow b$ and $b \rightarrow c$, then $a \rightarrow c$
    
* Creates a partial order
    * Not all events are related by _Happens-Before_

* Following events
    * Events within a process are all related by _Happens-Before_
    * Events between processes are related by following send->receive messages and linear events within processes.
    
![Lamport Timestamp Causality](img/lamport_causality.png)

### Assigning Timestamps

* Rules
    * Each process has a incrementing int counter, a "clock", starting at zero
    * For every **send** or **instruction**, increment the clock and assign the even the counter's timestamp
    * **Sends** carry their timestamp
    * **Receives** update the counter by 
    
    `max(local clock counter, message timestamp) + 1`
 
* Events without causal paths are _concurrent_ events
    * No guarantees as to ordering
    * Timestamps taken outside of a $\rightarrow$ relationship are not guaranteed to be ordered

### Distinguishing Concurrent From Causal

* Each process can keep tabs on its knowledge of other processes clocks
* For instance, in a vector $V_{i}$
* Known as _Vector Clocks_
* How they work
    * Local instructions or SEND events update Process _i_'s _i_ element in the local vector clock
    * SEND now carries the entire vector clock
    * RECEIVES now do two things
        * Update local entry in vector clock by one
        * Take max(sender, local) for each value in the local and received vector clocks

![Vector Clock Example](img/vector_clock_ex.png)

Elements are _concurrent_ if we cannot uniformly say that one vector clock is <= to another. For example, (2,2,0) ||| (1,1,3)